In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/*")

In [4]:
orders_rdd.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

Count the orders under each status

In [5]:
map_rdd=orders_rdd.map(lambda x: (x.split(",")[3],1))

In [6]:
map_rdd.take(10)

[('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('COMPLETE', 1),
 ('CLOSED', 1),
 ('COMPLETE', 1),
 ('COMPLETE', 1),
 ('COMPLETE', 1),
 ('PROCESSING', 1),
 ('PENDING_PAYMENT', 1),
 ('PENDING_PAYMENT', 1)]

In [7]:
reduced_rdd=map_rdd.reduceByKey(lambda x,y :x+y)

In [8]:
reduced_rdd.collect()

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899),
 ('PROCESSING', 8275),
 ('PAYMENT_REVIEW', 729),
 ('PENDING', 7610),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558)]

In [9]:
reduced_rdd_sorted=reduced_rdd.sortBy(lambda x:x[1],False)

In [10]:
reduced_rdd_sorted.collect()

[('COMPLETE', 22899),
 ('PENDING_PAYMENT', 15030),
 ('PROCESSING', 8275),
 ('PENDING', 7610),
 ('CLOSED', 7556),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558),
 ('CANCELED', 1428),
 ('PAYMENT_REVIEW', 729)]

Remove false for the Ascending order

Top 20 customers

In [11]:
customers_mapped=orders_rdd.map(lambda x: (x.split(",")[2],1))

In [12]:
customers_mapped.take(5)

[('11599', 1), ('256', 1), ('12111', 1), ('8827', 1), ('11318', 1)]

In [13]:
customers_reduced=customers_mapped.reduceByKey(lambda x,y: x+y)

In [14]:
customers_reduced.take(20)

[('256', 10),
 ('12111', 6),
 ('11318', 6),
 ('7130', 7),
 ('2911', 6),
 ('5657', 12),
 ('9149', 4),
 ('9842', 7),
 ('7276', 5),
 ('9488', 7),
 ('2711', 3),
 ('333', 6),
 ('656', 5),
 ('6983', 6),
 ('4189', 3),
 ('4840', 2),
 ('5863', 6),
 ('8214', 5),
 ('7776', 8),
 ('1549', 4)]

In [15]:
customers_sorted=customers_reduced.sortBy(lambda x:x[1],False)

In [16]:
customers_sorted.take(20)

[('5897', 16),
 ('6316', 16),
 ('12431', 16),
 ('569', 16),
 ('4320', 15),
 ('221', 15),
 ('5624', 15),
 ('5283', 15),
 ('12284', 15),
 ('5654', 15),
 ('11689', 14),
 ('5821', 14),
 ('791', 14),
 ('4517', 14),
 ('3708', 14),
 ('8652', 14),
 ('1940', 14),
 ('10591', 14),
 ('3710', 14),
 ('4249', 14)]

Distinct count of customers who placed atleast one order

In [17]:
distinct_customers=orders_rdd.map(lambda x: x.split(",")[2]).distinct()

In [18]:
distinct_customers.count()

12405

customers having more number of closed orders

In [19]:
filtered_orders=orders_rdd.filter(lambda x: x.split(",")[3] == "CLOSED")

In [20]:
filtered_orders.take(20)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '12,2013-07-25 00:00:00.0,1837,CLOSED',
 '18,2013-07-25 00:00:00.0,1205,CLOSED',
 '24,2013-07-25 00:00:00.0,11441,CLOSED',
 '25,2013-07-25 00:00:00.0,9503,CLOSED',
 '37,2013-07-25 00:00:00.0,5863,CLOSED',
 '51,2013-07-25 00:00:00.0,12271,CLOSED',
 '57,2013-07-25 00:00:00.0,7073,CLOSED',
 '61,2013-07-25 00:00:00.0,4791,CLOSED',
 '62,2013-07-25 00:00:00.0,9111,CLOSED',
 '87,2013-07-25 00:00:00.0,3065,CLOSED',
 '90,2013-07-25 00:00:00.0,9131,CLOSED',
 '101,2013-07-25 00:00:00.0,5116,CLOSED',
 '116,2013-07-26 00:00:00.0,8763,CLOSED',
 '129,2013-07-26 00:00:00.0,9937,CLOSED',
 '133,2013-07-26 00:00:00.0,10604,CLOSED',
 '191,2013-07-26 00:00:00.0,16,CLOSED',
 '201,2013-07-26 00:00:00.0,9055,CLOSED',
 '211,2013-07-26 00:00:00.0,10372,CLOSED']

In [21]:
filtered_mapped=filtered_orders.map(lambda x: (x.split(",")[2],1))

In [22]:
filtered_mapped.take(10)

[('11599', 1),
 ('8827', 1),
 ('1837', 1),
 ('1205', 1),
 ('11441', 1),
 ('9503', 1),
 ('5863', 1),
 ('12271', 1),
 ('7073', 1),
 ('4791', 1)]

In [23]:
filtered_aggregated=filtered_mapped.reduceByKey(lambda x,y: x+y)

In [24]:
filtered_aggregated.take(20)

[('3159', 1),
 ('5834', 2),
 ('10173', 1),
 ('2101', 1),
 ('6000', 1),
 ('1352', 2),
 ('10142', 1),
 ('12210', 1),
 ('6018', 2),
 ('2252', 1),
 ('10290', 2),
 ('9117', 1),
 ('7600', 2),
 ('6482', 1),
 ('9420', 1),
 ('11673', 3),
 ('7435', 2),
 ('7879', 4),
 ('11153', 3),
 ('9771', 1)]

In [25]:
 filtered_aggregated_sort=filtered_aggregated.sortBy(lambda x:x[1],False)

In [26]:
filtered_aggregated_sort.take(20)

[('1833', 6),
 ('1363', 5),
 ('1687', 5),
 ('5493', 5),
 ('5011', 4),
 ('8974', 4),
 ('2321', 4),
 ('3736', 4),
 ('8368', 4),
 ('2236', 4),
 ('2403', 4),
 ('7879', 4),
 ('1764', 4),
 ('4588', 4),
 ('7948', 4),
 ('7850', 4),
 ('145', 4),
 ('4282', 4),
 ('9213', 4),
 ('3631', 4)]